In [ ]:
# FreqAI Colab Setup Notebook

This notebook sets up SSH server on Colab and provides connection details for remote backtest execution.

## Cell 1: Enable GPU and Install SSH

```python
# Check GPU
import subprocess
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print("✅ GPU Available!")
    print(result.stdout)
except:
    print("❌ GPU not found. Enable GPU: Runtime > Change runtime type > GPU (T4)")
    
# Install SSH server
!apt-get update -qq
!apt-get install -y openssh-server pwgen > /dev/null

# Generate random password
import secrets
import string
password = ''.join(secrets.choice(string.ascii_letters + string.digits) for _ in range(16))
print(f"\n🔑 SSH Password: {password}")

# Configure SSH
!mkdir -p /var/run/sshd
!echo "root:{password}" | chpasswd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!echo "Port 22" >> /etc/ssh/sshd_config

# Start SSH service
!service ssh restart

print("✅ SSH server started on port 22")
```

## Cell 2: Setup ngrok Tunnel (No Installation Required)

```python
# Download and setup ngrok
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz
!chmod +x ngrok

# Get ngrok token from user
import getpass
ngrok_token = getpass.getpass("Enter your ngrok auth token (get from https://dashboard.ngrok.com/get-started/your-authtoken): ")

# Authenticate ngrok
!./ngrok authtoken {ngrok_token}

# Start ngrok tunnel in background
import subprocess
import time
import requests

# Start ngrok
ngrok_process = subprocess.Popen(['./ngrok', 'tcp', '22'], 
                                  stdout=subprocess.PIPE, 
                                  stderr=subprocess.PIPE)
time.sleep(3)

# Get tunnel URL
try:
    response = requests.get('http://localhost:4040/api/tunnels')
    tunnels = response.json()
    if tunnels.get('tunnels'):
        tunnel_url = tunnels['tunnels'][0]['public_url']
        print(f"\n✅ Tunnel established!")
        print(f"🌐 Tunnel URL: {tunnel_url}")
        print(f"🔑 Password: {password}")
        print(f"\n📋 Save these for local connection:")
        print(f"   --tunnel-url {tunnel_url}")
        print(f"   --password {password}")
    else:
        print("❌ No tunnel found. Check ngrok logs.")
except Exception as e:
    print(f"❌ Error: {e}")
```

## Cell 3: Clone Repository and Install Dependencies

```python
import os

# Clone repository
print("📥 Cloning repository...")
!rm -rf freqai-futures-strategy
!git clone https://github.com/aminak58/freqai-futures-strategy.git
os.chdir('/content/freqai-futures-strategy')

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q ta-lib-binary
!pip install -q freqtrade[freqai]

print("✅ Setup complete!")
print(f"📂 Working directory: {os.getcwd()}")
```

## Cell 4: Upload Data (if needed)

```python
# Option 1: Mount Google Drive (if you have data there)
from google.colab import drive
drive.mount('/content/drive')

# Extract data if available
import zipfile
import os

drive_zip = '/content/drive/MyDrive/FreqAI/binance_data.zip'
if os.path.exists(drive_zip):
    print("📦 Extracting data from Drive...")
    with zipfile.ZipFile(drive_zip, 'r') as zip_ref:
        zip_ref.extractall('/content/freqai-futures-strategy/user_data/data/')
    print("✅ Data extracted!")
else:
    print("⚠️ No data found in Drive. Will download fresh data.")

# Option 2: Download fresh data
# Uncomment if needed:
# !freqtrade download-data \
#   --exchange binance \
#   --pairs BTC/USDT:USDT ETH/USDT:USDT SOL/USDT:USDT \
#   --timeframes 5m 15m 1h \
#   --days 90 \
#   --trading-mode futures \
#   --config config/config.json
```

## Cell 5: Keep Session Alive

```python
# This cell keeps the session alive
# Run this to prevent disconnection

import time
from IPython.display import clear_output

print("🔄 Keeping session alive...")
print("🔑 Password:", password)
print("🌐 Tunnel URL:", tunnel_url)
print("\n⏳ Session will auto-refresh every 5 minutes")
print("Press Stop to end session")

try:
    while True:
        time.sleep(300)  # 5 minutes
        clear_output(wait=True)
        print("🔄 Session alive...")
        print("🔑 Password:", password)
        print("🌐 Tunnel URL:", tunnel_url)
        print(f"⏰ Time: {time.strftime('%H:%M:%S')}")
except KeyboardInterrupt:
    print("\n✋ Session monitoring stopped")
```

---

## Usage from Local Machine

Once the notebook is running and you have the tunnel URL and password:

```powershell
# Install automation dependencies
pip install -r requirements-automation.txt

# Execute backtest
python tools/backtest_executor.py `
  --tunnel-url "tcp://0.tcp.ngrok.io:XXXXX" `
  --password "your_password_here" `
  --strategy FreqAIHybridStrategy `
  --timerange 20250901-20251012 `
  --pairs BTC/USDT:USDT
```

## Batch Execution

```python
from tools.backtest_executor import ColabBacktestExecutor, BacktestConfig

configs = [
    BacktestConfig(
        strategy="FreqAIHybridStrategy",
        timerange="20250901-20251012",
        pairs=["BTC/USDT:USDT"]
    ),
    BacktestConfig(
        strategy="FreqAIHybridStrategy",
        timerange="20250801-20250831",
        pairs=["ETH/USDT:USDT"]
    ),
]

executor = ColabBacktestExecutor(
    tunnel_url="tcp://0.tcp.ngrok.io:XXXXX",
    password="your_password_here"
)

try:
    executor.connect()
    results = executor.execute_batch(configs)
    
    for result in results:
        print(f"Profit: {result['results']['total_profit_abs']} USDT")
        print(f"Sharpe: {result['results']['sharpe_ratio']}")
finally:
    executor.disconnect()
```